In [1]:
import numpy as np

from astropy.io import fits

import dysh.util as util
from dysh.util import sq_weighted_avg
from dysh.fits.gbtfitsload import GBTFITSLoad

In [2]:
# Point it to the testdata directory on the root dysh directory.
testdata    = util.get_project_testdata()
# The folder with the SDFITS files.
data_path   = testdata/"TGBT21A_501_11/"
data_file   = data_path/"TGBT21A_501_11_ifnum_0_int_0-2.fits"
gbtidl_file = data_path/"TGBT21A_501_11_ifnum_0_int_0-2_getps_152_plnum_0.fits"

In [3]:
# Generate the dysh result.
sdf = GBTFITSLoad(data_file)
# psscan is a ScanList.
psscan = sdf.getps(152, plnum=0)
assert len(psscan) == 1
psscan.calibrate()
# psscan_tavg is a list.
psscan_tavg = psscan.timeaverage(weights="tsys")
assert len(psscan_tavg) == 1

FILE  /home/sandboxes/psalas/Dysh/dysh/testdata/TGBT21A_501_11/TGBT21A_501_11_ifnum_0_int_0-2.fits
FILE  /home/sandboxes/psalas/Dysh/dysh/testdata/TGBT21A_501_11/TGBT21A_501_11_ifnum_0_int_0-2.fits


In [4]:
# Load the GBTIDL result.
hdu = fits.open(gbtidl_file)
table = hdu[1].data

In [5]:
assert psscan_tavg[0].meta["EXPOSURE"] == table["EXPOSURE"][0]

In [6]:
assert psscan_tavg[0].meta["TSYS"] == table["TSYS"][0]

AssertionError: 

In [7]:
assert psscan[0].exposure[0] == 0.9758745431900024

In [8]:
assert abs(psscan[0].delta_freq[0]) == 715.2557373046875000

In [9]:
assert psscan[0]._tsys[0] == 17.2400033063068747

In [10]:
assert psscan[0].exposure[0] * abs(psscan[0].delta_freq[0]) / psscan[0]._tsys[0]**2 == 2.3484463846686591815

In [11]:
from dysh.spectra.core import tsys_weight
assert tsys_weight(psscan[0].exposure[0], abs(psscan[0].delta_freq[0]), psscan[0]._tsys[0]) == 2.3484463846686591815

AssertionError: 

In [19]:
foo = psscan[0].exposure[0]*abs(psscan[0].delta_freq[0])*np.power(psscan[0]._tsys[0], -2.)
bar = psscan[0].exposure[0]*abs(psscan[0].delta_freq[0])/psscan[0]._tsys[0]**2.

In [23]:
foo?

Type:        float64
String form: 2.3484463846686587
File:        /home/sandboxes/psalas/Dysh/py3.11-dysh-note/lib/python3.11/site-packages/numpy/__init__.py
Docstring:  
Double-precision floating-point number type, compatible with Python `float`
and C ``double``.

:Character code: ``'d'``
:Canonical name: `numpy.double`
:Alias: `numpy.float_`
:Alias on this platform (Linux x86_64): `numpy.float64`: 64-bit precision floating-point number type: sign bit, 11 bits exponent, 52 bits mantissa.

In [22]:
bar?

Type:        float64
String form: 2.348446384668659
File:        /home/sandboxes/psalas/Dysh/py3.11-dysh-note/lib/python3.11/site-packages/numpy/__init__.py
Docstring:  
Double-precision floating-point number type, compatible with Python `float`
and C ``double``.

:Character code: ``'d'``
:Canonical name: `numpy.double`
:Alias: `numpy.float_`
:Alias on this platform (Linux x86_64): `numpy.float64`: 64-bit precision floating-point number type: sign bit, 11 bits exponent, 52 bits mantissa.

In [12]:
gbtidl_exposures = np.array([0.9758745431900024,
                             0.9727186456420835,
                             0.9758745431900024])
gbtidl_delta_freq = np.array([715.2557373046875000,
                              715.2557373046875000,
                              715.2557373046875000])
gbtidl_tsys = np.array([17.2400033063068747,
                        17.1714040732369675,
                        17.1566370578116292])
gbtidl_tsyswt = np.array([2.3484463846686592,
                          2.3595923244465413,
                          2.3713246243635990])
gbtdil_tsys2 = np.array([297.2177140014719612,
                         294.8571178463791398,
                         294.3501951334752675])
gbtidl_tsys2_sum = np.array([697.99986590624030214,
                             1393.7424579849857764,
                             2091.7423238912260786]) 2091.742323891226078558
gbtidl_tsys2_sum_here = np.cumsum(gbtdil_tsys2 * gbtidl_tsyswt)

In [13]:
gbtidl_tsys2_sum_here

array([ 697.99986591, 1393.74245798, 2091.74232389])

In [14]:
tsys2_sum = np.cumsum(psscan[0]._tsys**2. * psscan[0]._tsys_weight)#.astype("d")

for i in range(3):
    assert psscan[0].exposure[i] == gbtidl_exposures[i]
    assert abs(psscan[0].delta_freq[i]) == gbtidl_delta_freq[i]
    assert psscan[0]._tsys[i] == gbtidl_tsys[i]
    assert psscan[0]._tsys_weight[i] == gbtidl_tsyswt[i]
    assert psscan[0]._tsys[i]**2. == gbtdil_tsys2[i]
    assert tsys2_sum[i] == gbtidl_tsys2_sum[i]
    assert gbtidl_tsys2_sum_here[i] == gbtidl_tsys2_sum[i]

AssertionError: 

In [69]:
np.cumsum(psscan[0]._tsys**2. * psscan[0]._tsys_weight)[-1]

2091.7423238912261827

In [23]:
tsys_avg = np.average(psscan[0]._tsys, weights=psscan[0]._tsys_weight)
assert tsys_avg == table["TSYS"][0]

In [29]:
tsys_avg = sq_weighted_avg(psscan[0]._tsys, axis=0, weights=psscan[0]._tsys_weight)
# assert tsys_avg == table["TSYS"][0]
tsys_avg

17.189252488996036704

In [32]:
assert tsys_avg.astype(float) == table["TSYS"][0]

In [33]:
table["TSYS"]

array([17.18925249], dtype='>f8')

In [30]:
table["TSYS"][0]

17.189252488996036

In [28]:
tsys_avg - table["TSYS"][0]

3.9898639947466563171e-16

In [24]:
tsys_avg - table["TSYS"][0]

-3.830662236955356681e-05

In [13]:
psscan_tavg[0].meta["TSYS"] - table["TSYS"][0]

3.9898639947466563171e-16

In [12]:
assert psscan[0]._tsys_weight[0] == 2.3484463846686591815

In [36]:
diff = psscan_tavg[0].flux.value.astype("d") - table["DATA"][0]
diff

array([-2.89176166e-09,  6.73606942e-09, -1.73440567e-08, ...,
        2.43532005e-08, -2.24887755e-08,  8.01526637e-09])

In [38]:
psscan_tavg[0].flux.value

array([ 0.39660504,  0.14270814, -0.22407945, ...,  0.23408304,
       -0.02871263, -0.19871183], dtype=float128)

In [37]:
table["DATA"]

array([[ 0.39660504,  0.14270814, -0.22407943, ...,  0.23408301,
        -0.02871261, -0.19871184]], dtype='>f4')

In [10]:
hdu_ = fits.open(data_file)
table_ = hdu_[1].data

In [26]:
mask_sig = ( table_["SCAN"] == 152 ) & ( table_["PLNUM"] == 0 )
mask_ref = ( table_["SCAN"] == 153 ) & ( table_["PLNUM"] == 0 )
mask_on  = ( table_["CAL"][mask_sig] == "T" )
mask_off = ( table_["CAL"][mask_ref] == "T" )
exp_sig  = table_["EXPOSURE"][mask_sig][::2] + table_["EXPOSURE"][mask_sig][1::2]
exp_ref  = table_["EXPOSURE"][mask_ref][::2] + table_["EXPOSURE"][mask_ref][1::2]
nsmooth  = 1.
exp = exp_sig * exp_ref * nsmooth / (exp_sig + exp_ref * nsmooth)

In [28]:
exp.sum()

2.924467732022088

In [8]:
psscan[0].exposure.sum()

2.924467732022088

In [16]:
table_["CAL"][mask]

chararray(['F', 'T', 'F', 'T', 'F', 'T', 'F', 'T', 'F', 'T', 'F', 'T'],
          dtype='<U1')

In [17]:
table_["DURATION"][mask][::2]

array([0.9982445, 0.9982445, 0.9982445, 0.9982445, 0.9982445, 0.9982445],
      dtype='>f8')

In [29]:
hdut = fits.open(f"{data_path}/test_exposure.fits")
tablet = hdut[1].data
tablet["EXPOSURE"]

array([2.92446773], dtype='>f8')

In [30]:
tablet["EXPOSURE"] - exp.sum()

array([0.])

In [31]:
psscan_tavg[0].meta["EXPOSURE"] - tablet["EXPOSURE"]

array([0.])

In [32]:
psscan_tavg[0].meta["EXPOSURE"] - table["EXPOSURE"]

array([1.68392938e-07])

In [78]:
hdu = fits.open(f"{data_path}/test_int_0.fits")
table = hdu[1].data

In [79]:
table["DATA"][0] - psscan[0].calibrated(0).flux.value.astype("d")

array([-7.45058060e-09,  0.00000000e+00, -7.45058060e-09, ...,
       -2.98023224e-08,  0.00000000e+00, -1.49011612e-08])

In [49]:
assert table["TSYS"] == psscan[0].calibrated(0).meta["TSYS"]
assert table["EXPOSURE"] == psscan[0].calibrated(0).meta["EXPOSURE"]
assert table["CDELT1"] == psscan[0].calibrated(0).meta["CDELT1"]

In [81]:
for i in range(len(table["DATA"][0])):
    try:
        assert table["DATA"][0][i] == psscan[0].calibrated(0).flux.value[i]
    except AssertionError:
        print(i)

0
2
3
7
8
11
12
14
15
22
24
25
27
28
29
31
34
35
37
44
49
52
54
57
58
59
65
66
69
71
72
74
75
77
80
86
88
101
102
103
104
105
109
114
115
117
118
122
125
127
128
129
132
133
134
142
147
148
149
150
151
153
160
162
165
166
167
168
170
172
174
176
182
184
185
186
189
191
195
197
201
203
205
206
208
209
216
217
221
222
223
226
231
233
237
241
245
246
250
251
254
257
264
265
267
270
271
277
278
279
280
282
284
285
292
294
295
297
298
300
301
305
309
315
324
325
328
331
336
344
346
347
348
350
351
352
357
360
361
368
370
371
375
379
382
383
384
386
389
391
398
400
405
407
408
410
413
415
416
417
418
419
424
425
426
428
429
431
435
439
440
445
446
447
449
450
451
452
454
455
458
460
462
463
466
468
470
471
476
477
479
481
482
483
484
485
494
495
497
500
502
507
512
514
516
518
523
524
525
528
531
536
537
540
541
544
546
554
555
561
562
565
569
572
573
574
575
576
578
583
586
587
595
596
597
598
601
602
607
609
610
612
615
616
617
620
624
626
627
632
633
634
639
643
644
648
651
654
658
662
66

KeyboardInterrupt: 

In [83]:
hdu = fits.open(f"{data_path}/test_gettp_0.fits")
table = hdu[1].data

In [108]:
gbtidl_data = table["DATA"][0]

In [107]:
dysh_data = sdf._sdf[0].rawspectra(psscan[0]._bintable_index)[psscan[0]._sigonrows][0]

In [112]:
np.nansum((gbtidl_data - dysh_data))

0.0

In [92]:
table_ = sdf._sdf[0]._hdu[1].data
mask = (table_["SCAN"] == 152) & \
       (table_["IFNUM"] == 0) & \
       (table_["PLNUM"] == 0) & \
       (table_["CAL"] == "T") & \
       (table_["SIG"] == "T")
mask.sum()


3

In [96]:
table_[mask]["DATA"][0]

array([3551465.8, 3179559.8, 3600287.5, ..., 3723257.8, 3600230.2,
       3435717. ], dtype='>f4')

In [95]:
sdf.summary()

,SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATIO
0,152,NGC2415,3784.0,OnOff,1,1.420406,1.420406,1,2,3,1,285.954501,42.093531
1,153,NGC2415,3784.0,OnOff,2,1.420406,1.420406,1,2,3,1,286.624729,41.579808


In [113]:
hdu = fits.open(f"{data_path}/test_gettp_intnum_0.fits")
table = hdu[1].data

In [114]:
tp = sdf.gettp(152)

In [118]:
np.nansum(tp[0].total_power(0).flux.value - table["DATA"])

0.0

In [24]:
hdu = fits.open(f"{data_path}/test_getps_intnum_0.fits")
table = hdu[1].data

In [25]:
ps = sdf.getps(152)

FILE  /home/sandboxes/psalas/Dysh/dysh/testdata/TGBT21A_501_11/TGBT21A_501_11_ifnum_0_int_0-2.fits
FILE  /home/sandboxes/psalas/Dysh/dysh/testdata/TGBT21A_501_11/TGBT21A_501_11_ifnum_0_int_0-2.fits


In [28]:
(ps[0].calibrated(0).flux.value == table["DATA"][1])

array([False,  True, False, ..., False,  True, False])

In [32]:
assert ps[0]._tsys[0] == table["TSYS"][0]
assert ps[0]._tsys[0] == table["TSYS"][1]

In [29]:
table["DATA"][0]

array([ 0.09754237, -0.4535061 ,  0.06858955, ...,  0.48103508,
       -0.21121033, -0.23867549], dtype='>f4')

In [30]:
table["DATA"][1]

array([ 0.09754237, -0.4535061 ,  0.06858955, ...,  0.48103508,
       -0.21121033, -0.23867549], dtype='>f4')